<a href="https://colab.research.google.com/github/HarikrishnanK9/DataSymphony/blob/main/LLM_word_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [3]:
with open('/content/IndiaUS.txt','r',encoding='utf-8') as myfile:
  mytext = myfile.read()

In [4]:
print(mytext)

Following a lavish state visit by Indian Prime Minister Narendra Modi to Washington, US President Joe Biden has called his country's partnership with India among the "most consequential in the world". The BBC's Vikas Pandey and Soutik Biswas explore the factors that contribute to the visit's potential in strengthening the ties between the two nations.

The US's relationship with India - the world's most populous country - is "stronger, closer and more dynamic than any time in history", Mr Biden said at the completion of a pomp-filled state visit by Mr Modi to the White House.

The remark may not be an exaggeration. "This summit suggests that the relationship has been transformed. It underscores just how broad and deep it has become in a relatively short time," says Michael Kugelman of The Wilson Center, an American think-tank.

A key reason is that Washington is keen to draw India closer so that it can act as a counterbalance to China's growing influence in the Indo-Pacific. India-US t

In [5]:
mytokenizer=Tokenizer()

In [19]:
mytokenizer.fit_on_texts([mytext])
total_words = len(mytokenizer.word_index)+1
total_words

599

In [20]:
mytokenizer.word_index

{'the': 1,
 'to': 2,
 'in': 3,
 'a': 4,
 'and': 5,
 'india': 6,
 'of': 7,
 'mr': 8,
 'us': 9,
 'is': 10,
 'modi': 11,
 'has': 12,
 'that': 13,
 'biden': 14,
 'as': 15,
 'with': 16,
 'not': 17,
 "india's": 18,
 'for': 19,
 'but': 20,
 'on': 21,
 'washington': 22,
 'an': 23,
 'it': 24,
 'says': 25,
 'will': 26,
 'are': 27,
 'indian': 28,
 'prime': 29,
 'minister': 30,
 'this': 31,
 'have': 32,
 'trade': 33,
 'visit': 34,
 'by': 35,
 'president': 36,
 'relationship': 37,
 'been': 38,
 'about': 39,
 'strategic': 40,
 'more': 41,
 'up': 42,
 'during': 43,
 'also': 44,
 'make': 45,
 'sirohi': 46,
 'china': 47,
 'air': 48,
 'force': 49,
 'state': 50,
 'his': 51,
 'ties': 52,
 'two': 53,
 'at': 54,
 'white': 55,
 'house': 56,
 'be': 57,
 'lot': 58,
 'ms': 59,
 'technology': 60,
 'jet': 61,
 'semiconductor': 62,
 'now': 63,
 'first': 64,
 'was': 65,
 'global': 66,
 'one': 67,
 'he': 68,
 'without': 69,
 'they': 70,
 'narendra': 71,
 'joe': 72,
 'world': 73,
 'potential': 74,
 'how': 75,
 'becom

In [21]:
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(mytokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
my_input_sequences=[]

for line in mytext.split('\n'):
  token_list = mytokenizer.texts_to_sequences([line])[0]
  # print(token_list)
  for i in range(1,len(token_list)):
    #n-gram sequences
    my_ngram_sequence = token_list[:i+1]
    my_input_sequences.append(my_ngram_sequence)

In [23]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
max_sequence_len

83

In [24]:
# """
# [1,3,4,5]
# [2,3,4,5]
# [14,5,2]
# """

In [25]:
input_sequence = np.array(pad_sequences(my_input_sequences,maxlen = max_sequence_len,padding='pre')) #default padding pre thanneyanu

In [26]:
input_sequence

array([[  0,   0,   0, ...,   0,  99,   4],
       [  0,   0,   0, ...,  99,   4, 177],
       [  0,   0,   0, ...,   4, 177,  50],
       ...,
       [  0,   0,   0, ..., 176, 598,  25],
       [  0,   0,   0, ..., 598,  25,  59],
       [  0,   0,   0, ...,  25,  59,  46]], dtype=int32)

In [27]:
x =input_sequence[:,:-1]
y = input_sequence[:,-1]

In [28]:
x

array([[  0,   0,   0, ...,   0,   0,  99],
       [  0,   0,   0, ...,   0,  99,   4],
       [  0,   0,   0, ...,  99,   4, 177],
       ...,
       [  0,   0,   0, ...,   5, 176, 598],
       [  0,   0,   0, ..., 176, 598,  25],
       [  0,   0,   0, ..., 598,  25,  59]], dtype=int32)

In [29]:
y

array([  4, 177,  50, ...,  25,  59,  46], dtype=int32)

In [30]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [31]:
model =Sequential()
model.add(Embedding(input_dim = total_words,output_dim=100,input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words,activation='softmax'))

In [32]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])


In [33]:
model.fit(x,y,epochs=50,verbose=1)

Epoch 1/50
43/43 [==============================] - 13s 131ms/step - loss: 6.1983 - accuracy: 0.0379
Epoch 2/50
43/43 [==============================] - 6s 129ms/step - loss: 5.7947 - accuracy: 0.0576
Epoch 3/50
43/43 [==============================] - 4s 86ms/step - loss: 5.7256 - accuracy: 0.0576
Epoch 4/50
43/43 [==============================] - 2s 44ms/step - loss: 5.6674 - accuracy: 0.0576
Epoch 5/50
43/43 [==============================] - 2s 47ms/step - loss: 5.5904 - accuracy: 0.0576
Epoch 6/50
43/43 [==============================] - 1s 33ms/step - loss: 5.4890 - accuracy: 0.0591
Epoch 7/50
43/43 [==============================] - 1s 20ms/step - loss: 5.3749 - accuracy: 0.0671
Epoch 8/50
43/43 [==============================] - 1s 34ms/step - loss: 5.2349 - accuracy: 0.0875
Epoch 9/50
43/43 [==============================] - 1s 24ms/step - loss: 5.0340 - accuracy: 0.1036
Epoch 10/50
43/43 [==============================] - 2s 50ms/step - loss: 4.8080 - accuracy: 0.1174
Epoch 

In [34]:
model.save('word_prediction.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
!cp word_prediction.h5  /content/drive/MyDrive/LLM

In [36]:
!cp tokenizer.pickle /content/drive/MyDrive/LLM

# Prediction

In [37]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

In [38]:
with open('/content/tokenizer.pickle','rb') as handle:
  mytokenizzer = pickle.load(handle)

  model = tf.keras.models.load_model('/content/word_prediction.h5')


In [42]:
# input_text = 'narendra modi'
# predict_next_words = 3

# for i in range(predict_next_words):
#   token_list = mytokenizer.texts_to_sequences([input_text])[0]
#   token_list = pad_sequences([token_list],maxlen=model.input_shape[1],padding='pre')
#   predicted = np.argmax(model.predict(token_list))
#   print(predicted)

#   output_word = mytokenizer.index_word(predicted)

# input_text += " "+ output_word
# print(input_text)

In [43]:
input_text = 'narendra modi'
predict_next_words = 3

for i in range(predict_next_words):
  token_list = mytokenizer.texts_to_sequences([input_text])[0]
  token_list = pad_sequences([token_list], maxlen=model.input_shape[1], padding='pre')
  predicted = np.argmax(model.predict(token_list))
  print(predicted)

  output_word = mytokenizer.index_word[predicted]

  input_text += " " + output_word

print(input_text)


1/1 [==============================] - 0s 59ms/step
2
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 18ms/step
55
narendra modi to the white
